## Part 2 : REINFORCE algorithm

In [1]:
import gymnasium as gym
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import pygame
from collections import deque
from tqdm import *

## Description

The system consists of two links connected linearly to form a chain, with one end of the chain fixed. The joint between the two links is actuated. The goal is to apply torques on the actuated joint to swing the free end of the linear chain above a given height while starting from the initial state of hanging downwards.

## Action Space
The action is discrete, deterministic, and represents the torque applied on the actuated joint between the two links.
 | Num | Action                                | Unit         |
    |-----|---------------------------------------|--------------|
    | 0   | apply -1 torque to the actuated joint | torque (N m) |
    | 1   | apply 0 torque to the actuated joint  | torque (N m) |
    | 2   | apply 1 torque to the actuated joint  | torque (N m) 

## Observation Space
The observation is a ndarray with shape (6,) that provides information about the two rotational joint angles as well as their angular velocities.

    | Num | Observation                  | Min                 | Max               |
    |-----|------------------------------|---------------------|-------------------|
    | 0   | Cosine of `theta1`           | -1                  | 1                 |
    | 1   | Sine of `theta1`             | -1                  | 1                 |
    | 2   | Cosine of `theta2`           | -1                  | 1                 |
    | 3   | Sine of `theta2`             | -1                  | 1                 |
    | 4   | Angular velocity of `theta1` | ~ -12.567 (-4 * pi) | ~ 12.567 (4 * pi) |
    | 5   | Angular velocity of `theta2` | ~ -28.274 (-9 * pi) | ~ 28.274 (9 * pi) |

where

theta1 is the angle of the first joint, where an angle of 0 indicates the first link is pointing directly downwards.

theta2 is relative to the angle of the first link. An angle of 0 corresponds to having the same angle between the two links.

The angular velocities of theta1 and theta2 are bounded at ±4π, and ±9π rad/s respectively. A state of [1, 0, 1, 0, ..., ...] indicates that both links are pointing downwards.

## Rewards
The goal is to have the free end reach a designated target height in as few steps as possible, and as such all steps that do not reach the goal incur a reward of -1. Achieving the target height results in termination with a reward of 0. The reward threshold is -100.

## Starting State
Each parameter in the underlying state (theta1, theta2, and the two angular velocities) is initialized uniformly between -0.1 and 0.1. This means both links are pointing downwards with some initial stochasticity.

## Episode End
The episode ends if one of the following occurs:

1. Termination: The free end reaches the target height, which is constructed as: -cos(theta1) - cos(theta2 + theta1) > 1.0

2. Truncation: Episode length is greater than 500 (200 for v0)

In [2]:
env = gym.make('Acrobot-v1')

Number of possible actions: Box(-2.0, 2.0, (1,), float32) that correspond to the torque applied to the free end of the pendulum (positive counter-clock wise)
Number of states: Box([-1. -1. -8.], [1. 1. 8.], (3,), float32) that correspond to the x-y coordinates (Cartesian basis) and the angular velocity of the free end of the pendulum


In [ ]:
# To change the dynamics as described above
env.env.book_or_nips = 'nips'

In [ ]:
n_actions = env.action_space.n
shape_states = env.observation_space.shape
print(n_actions)
print(shape_states)

In [4]:
def theta_init(x):
    theta=[0]*len(x)
    return np.array(theta)

def policy(action,state,theta):
    p=state @ theta
    q=1/(1+np.exp(-p))
    if action==0:
        return 1-q
    else : 
        return q
def gradient_function(action,state,theta):
    if action==0:
        return -state*policy(1,state,theta)
    else :
        return state*policy(0,state,theta)

In [4]:
def action(state,theta):
    if state[1]>=0:
        action=np.random.uniform(-2,0)
    else :
        action=np.random.uniform(0,2)
    return action

def gradient_function(action,state,theta):
    if action==0:
        return -state*policy(1,state,theta)
    else :
        return state*policy(0,state,theta)

In [8]:
def reinforce(theta_0,lr,gamma,n_episode):
    env = gym.make("Pendulum-v1", render_mode="human")
    theta=theta_0
    
    for i in range(n_episode):
        X=[] #list of states
        A=[] #list of actions
        R=[] #list of rewards
        x,_=env.reset()
        n_move = 0 
        terminated=False
        while not terminated: #episode to fill the lists
            if n_move > 500:
                env.close()
                raise Exception("Too many attempts, failed")
            n_move += 1
            X.append(x)
            
            u= np.random.uniform(0,1)
            if u<=policy(1,x,theta):
                action=1
            else :
                action=0
            A.append(action)
            x, r, terminated, truncated, info = env.step([action])
            R.append(r)
        
        print("Total rewards :",np.sum(R))
        n=0 
        while n<n_move: #list run for the adjustment of theta
          
            G=0
            for i in range(n+1,n_move):
                G=G+gamma**(i-n-1)*R[i]
                
            grad=gradient_function(A[n],X[n],theta)
            theta=theta+lr*gamma**n*G*grad
            
            n += 1
    env.close()
    return theta

In [6]:
lr=0.001
n_episode=100
gamma=1
theta_0=[0,0,0]

In [9]:
reinforce(theta_0,lr,gamma,n_episode)

Exception: Too many attempts, failed

In [ ]:
for i in range(n_episode):
    env = gym.make("CartPole-v1", render_mode = "human")
    x,info=env.reset()
    n_move=0
    action = env.action_space.sample()
    while not terminated:
        if n_move > 10000:
            raise Exception("Too many attempts, failed")
        n_move += 1
        X.append(x)
        policy=policyparam(action,x,theta)
        action = np.argmax(policy)
        A.append(action)
        x, reward, terminated, truncated, info = env.step(action)
        print(reward)
        R.append(reward)
    env.close()